In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import os
import cv2
import numpy
import imageio
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, generic_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from keras import backend as K
import sys
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [0]:
image_rows, image_columns, image_depth = 64, 64, 18

In [0]:
training_list = []
painpath = '/content/drive/My Drive/Final Training Data/pain'
Nopainpath = '/content/drive/My Drive/Final Training Data/Nopain'

In [0]:
directorylisting = os.listdir(painpath)

In [0]:
for video in os.listdir(painpath):
  videopath = os.path.join(painpath, video)
  frames = []
  for image_path in os.listdir(videopath):
        input_path = os.path.join(videopath, image_path)
        image = cv2.imread(input_path)
        imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
        grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
        frames.append(grayimage)
  frames = numpy.asarray(frames)
  videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
  training_list.append(videoarray)

In [0]:
len(training_list)

25

In [0]:
directorylisting = os.listdir(Nopainpath)

In [0]:
for video in os.listdir(Nopainpath):
  videopath = os.path.join(Nopainpath, video)
  frames = []
  for image_path in os.listdir(videopath):
        input_path = os.path.join(videopath, image_path)
        image = cv2.imread(input_path)
        imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
        grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
        frames.append(grayimage)
  frames = numpy.asarray(frames)
  videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
  training_list.append(videoarray)

In [0]:
len(training_list)

50

In [0]:
training_list = numpy.asarray(training_list)
trainingsamples = len(training_list)
traininglabels = numpy.zeros((trainingsamples, ), dtype = int)

In [0]:
traininglabels[0:25] = 0
traininglabels[25:50] = 1

In [0]:
traininglabels = np_utils.to_categorical(traininglabels, 2)

In [0]:
training_data = [training_list, traininglabels]
(trainingframes, traininglabels) = (training_data[0], training_data[1])
training_set = numpy.zeros((trainingsamples, 1, image_rows, image_columns, image_depth))

In [0]:
training_set.shape

(50, 1, 64, 64, 18)

In [0]:
for h in range(trainingsamples):
    training_set[h][0][:][:][:] = trainingframes[h, :, :, :]

In [0]:
training_set = training_set.astype('float32')
training_set -= numpy.mean(training_set)
training_set /= numpy.max(training_set)

In [0]:
model = Sequential()
model.add(Convolution3D(32, (3, 3, 15), input_shape=(1, image_rows, image_columns, image_depth), activation='relu', trainable = False))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, init='normal', activation='relu', trainable = True))
model.add(Dropout(0.5))
model.add(Dense(3, init='normal', trainable = True))

W0403 03:44:12.019201 140066983888768 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0403 03:44:12.037456 140066983888768 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0403 03:44:12.044023 140066983888768 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0403 03:44:12.057527 140066983888768 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0403 03:44:12.058690 140066983888768 module_wrapper

In [0]:
model.load_weights('/content/drive/My Drive/Micro-exp dataset/dataset_and_weights (1)/dataset_and_weights/weights_microexpstcnn/weights-improvement-40-0.69.hdf5')

In [0]:
model.add(Dense(2, init='normal'))
model.add(Activation('softmax'))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_initializer="normal")`
  """Entry point for launching an IPython kernel.


In [0]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
model.summary()

W0403 03:44:33.741144 140066983888768 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0403 03:44:33.750425 140066983888768 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 32, 62, 62, 4)     4352      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 32, 20, 20, 1)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 20, 20, 1)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1638528   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
__________

In [0]:
train_images, validation_images, train_labels, validation_labels =  train_test_split(training_set, traininglabels, test_size=0.2, random_state=4)

In [0]:
hist = model.fit(train_images, train_labels, validation_data = (validation_images, validation_labels), nb_epoch = 35, shuffle=True)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.
W0403 03:44:38.163868 140066983888768 deprecation.py:323] From /tensorflow-1.15.2/python2.7/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0403 03:44:38.258833 140066983888768 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0403 03:44:38.288322 140066983888768 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 40 samples, validate on 10 samples
Epoch 1/35
40/40 [==============================] - 2s 45ms/step - loss: 0.7260 - acc: 0.4750 - val_loss: 0.7029 - val_acc: 0.6000
Epoch 2/35
40/40 [==============================] - 1s 29ms/step - loss: 0.7052 - acc: 0.4500 - val_loss: 0.6883 - val_acc: 0.4000
Epoch 3/35
40/40 [==============================] - 1s 29ms/step - loss: 0.6805 - acc: 0.5500 - val_loss: 0.6733 - val_acc: 0.4000
Epoch 4/35
40/40 [==============================] - 1s 31ms/step - loss: 0.6618 - acc: 0.5500 - val_loss: 0.6596 - val_acc: 0.4000
Epoch 5/35
40/40 [==============================] - 1s 29ms/step - loss: 0.6464 - acc: 0.6000 - val_loss: 0.6476 - val_acc: 0.4000
Epoch 6/35
40/40 [==============================] - 1s 29ms/step - loss: 0.6238 - acc: 0.6000 - val_loss: 0.6355 - val_acc: 0.4000
Epoch 7/35
40/40 [==============================] - 1s 29ms/step - loss: 0.5994 - acc: 0.6250 - val_loss: 0.6216 - val_acc: 0.4000
Epoch 8/35
40/40 [=====================

In [0]:
model.save('/content/drive/My Drive/Research project/Pre-trained_model_transfer.h5')

In [0]:
predictions = model.predict(validation_images)
predictions_labels = numpy.argmax(predictions, axis=1)
validation_labels = numpy.argmax(validation_labels, axis=1)
cfm = confusion_matrix(validation_labels, predictions_labels)
print (cfm)

[[5 1]
 [0 4]]
